# TF2 에서 지원하는 모델 저장/로드 연습

* model.save, save_model
* from_config, to_config
* model_from_json, to_json
* save_weights, load_weights

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# SavedModel 로 모델 전체 저장

In [2]:
def get_model():
    # Create a simple model.
    inputs = keras.Input(shape=(32,))
    outputs = keras.layers.Dense(1)(inputs)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="adam", loss="mean_squared_error")
    return model

model = get_model()

# Train the model.
test_input = np.random.random((128, 32))
test_target = np.random.random((128, 1))
model.fit(test_input, test_target)

# Calling `save('my_model')` creates a SavedModel folder `my_model`.
model.save("/tmp/logs/my_model")

# It can be used to reconstruct the model identically.
reconstructed_model = keras.models.load_model("/tmp/logs/my_model")

# Let's check:
np.testing.assert_allclose(
    model.predict(test_input), reconstructed_model.predict(test_input)
)

# The reconstructed model is already compiled and has retained the optimizer
# state, so training can resume:
reconstructed_model.fit(test_input, test_target)

4/4 [==============================] - 0s 418us/step - loss: 0.8764


# 사용자 정의 모델 사용하는 SavedModel 로딩시에 원래 정의 class가 필요없다.

* 로더는 기존 모델처럼 작동하는 새 모델 클래스를 동적으로 만든다.

In [3]:
class CustomModel(keras.Model):
    def __init__(self, hidden_units):
        super(CustomModel, self).__init__()
        self.dense_layers = [keras.layers.Dense(u) for u in hidden_units]

    def call(self, inputs):
        x = inputs
        for layer in self.dense_layers:
            x = layer(x)
        return x
    
model = CustomModel([16, 16, 10])
# Build the model by calling it
input_arr = tf.random.uniform((1, 5))
outputs = model(input_arr)
model.save("/tmp/logs/my_model")   

# Delete the custom-defined model class to ensure that the loader does not have
# access to it.
del CustomModel

loaded = keras.models.load_model("/tmp/logs/my_model")
np.testing.assert_allclose(loaded(input_arr), outputs)

print("Original model:", model)
print("Loaded model:", loaded)

Original model: <__main__.CustomModel object at 0x7fd2140b11d0>
Loaded model: <tensorflow.python.keras.saving.saved_model.load.CustomModel object at 0x7fd1f4774d68>


# 아키텍처 저장

## 레이어 저장

In [4]:
layer = keras.layers.Dense(3, activation="relu")
layer_config = layer.get_config()
print(f'layer config: {layer_config}\n')

new_layer = keras.layers.Dense.from_config(layer_config)
print(f'new_layer: {new_layer}')

layer config: {'name': 'dense_4', 'trainable': True, 'dtype': 'float32', 'units': 3, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}

new_layer: <tensorflow.python.keras.layers.core.Dense object at 0x7fd25a45d550>


## Sequential model 예제

In [7]:
model = keras.Sequential([keras.Input((32,)), keras.layers.Dense(1)])
model_config = model.get_config()
print(f'model config: {model_config}\n')

new_model = keras.Sequential.from_config(model_config)
print(f'new_model: {new_model}')

model config: {'name': 'sequential_2', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 32), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_4'}}, {'class_name': 'Dense', 'config': {'name': 'dense_7', 'trainable': True, 'dtype': 'float32', 'units': 1, 'activation': 'linear', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}}]}

new_model: <tensorflow.python.keras.engine.sequential.Sequential object at 0x7fd214152860>


## Functional model 예제

In [8]:
inputs = keras.Input((32,))
outputs = keras.layers.Dense(1)(inputs)
model = keras.Model(inputs, outputs)
config = model.get_config()
print(f'model config: {config}\n')

new_model = keras.Model.from_config(config)
print(f'new_model: {new_model}')

model config: {'name': 'functional_3', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 32), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_5'}, 'name': 'input_5', 'inbound_nodes': []}, {'class_name': 'Dense', 'config': {'name': 'dense_8', 'trainable': True, 'dtype': 'float32', 'units': 1, 'activation': 'linear', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}, 'name': 'dense_8', 'inbound_nodes': [[['input_5', 0, 0, {}]]]}], 'input_layers': [['input_5', 0, 0]], 'output_layers': [['dense_8', 0, 0]]}

new_model: <tensorflow.python.keras.engine.functional.Functional object at 0x7fd1f46c0710>


## Json으로 저장/조회

In [11]:
model = keras.Sequential([keras.Input((32,)), keras.layers.Dense(1)])
json_config = model.to_json()
print(f'model json config: {json_config}\n')

new_model = keras.models.model_from_json(json_config)
print(f'new_model: {new_model}')

model json config: {"class_name": "Sequential", "config": {"name": "sequential_5", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 32], "dtype": "float32", "sparse": false, "ragged": false, "name": "input_8"}}, {"class_name": "Dense", "config": {"name": "dense_11", "trainable": true, "dtype": "float32", "units": 1, "activation": "linear", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}]}, "keras_version": "2.4.0", "backend": "tensorflow"}

new_model: <tensorflow.python.keras.engine.sequential.Sequential object at 0x7fd1f463cf98>


## 사용자 정의 모델 

* get_config, from_config define
* custom_object 정의
* custom_object_scope 내에서 복원

In [18]:
class CustomLayer(keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(CustomLayer, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

    def get_config(self):
        config = super(CustomLayer, self).get_config()
        config.update({"units": self.units})
        return config
    
def custom_activation(x):
    return tf.nn.tanh(x) ** 2

# Make a model with the CustomLayer and custom_activation
inputs = keras.Input((32,))
x = CustomLayer(32)(inputs)
outputs = keras.layers.Activation(custom_activation)(x)
model = keras.Model(inputs, outputs)

# Retrieve the config
config = model.get_config()
print(f'model config: {config}\n')

# At loading time, register the custom objects
custom_objects = {'CustomLayer': CustomLayer, 'custom_activation': custom_activation}
with keras.utils.custom_object_scope(custom_objects):
    new_model = keras.Model.from_config(config)

model config: {'name': 'functional_15', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 32), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'input_14'}, 'name': 'input_14', 'inbound_nodes': []}, {'class_name': 'CustomLayer', 'config': {'name': 'custom_layer_5', 'trainable': True, 'dtype': 'float32', 'units': 32}, 'name': 'custom_layer_5', 'inbound_nodes': [[['input_14', 0, 0, {}]]]}, {'class_name': 'Activation', 'config': {'name': 'activation_5', 'trainable': True, 'dtype': 'float32', 'activation': 'custom_activation'}, 'name': 'activation_5', 'inbound_nodes': [[['custom_layer_5', 0, 0, {}]]]}], 'input_layers': [['input_14', 0, 0]], 'output_layers': [['activation_5', 0, 0]]}



# 모델의 가중치 값만 저장 및 로딩 

In [20]:
def create_layer():
    layer = keras.layers.Dense(64, activation="relu", name="dense_2")
    layer.build((None, 784))
    return layer

layer_1 = create_layer()
layer_2 = create_layer()

# copy weights
layer_2.set_weights(layer_1.get_weights())

In [23]:
# TF Checkpoint 형식으로 저장 후 로딩

sequential_model = keras.Sequential(
    [
        keras.Input(shape=(784,), name="digits"),
        keras.layers.Dense(64, activation="relu", name="dense_1"),
        keras.layers.Dense(64, activation="relu", name="dense_2"),
        keras.layers.Dense(10, name="predictions"),
    ]
)

sequential_model.save_weights('ckpt')
load_status = sequential_model.load_weights('ckpt')
load_status.assert_consumed()

# 전이학습 예제

In [25]:
def create_functional_model():
    inputs = keras.Input(shape=(784,), name="digits")
    x = keras.layers.Dense(64, activation="relu", name="dense_1")(inputs)
    x = keras.layers.Dense(64, activation="relu", name="dense_2")(x)
    outputs = keras.layers.Dense(10, name="predictions")(x)
    return keras.Model(inputs=inputs, outputs=outputs, name="3_layer_mlp")

functional_model = create_functional_model()
functional_model.save_weights("pretrained_weights.h5")

In [28]:
# 로딩해서 별도 모델에서 이용
pretrained_model = create_functional_model()
pretrained_model.load_weights('pretrained_weights.h5')

extracted_layers = pretrained_model.layers[:-1]
extracted_layers.append(keras.layers.Dense(5, name="dense_3"))
model = keras.Sequential(extracted_layers)
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 325       
Total params: 54,725
Trainable params: 54,725
Non-trainable params: 0
_________________________________________________________________
